In [1]:
!pip install --upgrade tensorflow sagemaker


  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached sagemaker-2.232.1-py3-none-any.whl.metadata (16 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.5.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached boto3-1.35.24-py3-none-any.whl.metadata (6.6 kB)
  Using cached sagemaker_core-1.0.7-py3-none-any.whl.metadata (4.9 kB)
  Using cached botocore-1.35.24-py3-none-any.whl.metadata (5.7 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached optree-0.12.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (47 kB)

In [2]:
import boto3
import numpy as np
import os
import pandas as pd
import re
import sagemaker
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.utils import S3DataConfig

import shutil
import tarfile
import tensorflow as tf
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Input, Dense

role = sagemaker.get_execution_role()
sm_session = sagemaker.Session()
bucket_name = sm_session.default_bucket()
prefix = "sagemaker/modelLSTM"
print(bucket_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2024-09-23 20:08:40.912599: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 20:08:40.915786: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 20:08:40.925802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 20:08:40.943483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 20:08:40.948684: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 20:08:40.961191: I tensorflow/core/platform/cpu_feature_gu

sagemaker-eu-west-3-024848443248


In [3]:
import os
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('urgency_cases_data_large_multiclass (2).csv')
data = df.drop(columns=['patient_id'])

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[['heart_rate', 'respiratory_rate', 'blood_pressure_diastolic','blood_pressure_systolic', 'oxygen_saturation', 'body_temperature']])
def create_sequences(dataa, target, time_steps=10, lead_time=10): 
    X, y = [], []
    for i in range(len(dataa) - time_steps - lead_time):
        X.append(dataa[i:(i + time_steps)])
        y.append(target[i + time_steps + lead_time])
    return np.array(X), np.array(y)

time_steps = 10
X, y = create_sequences(scaled_data, data['urgency'].values, time_steps)
y = to_categorical(y, num_classes=4)

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = Sequential()
model.add(LSTM(50, input_shape=(time_steps, X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=50, batch_size=42, validation_split=0.2, callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.2f}, Test Accuracy: {accuracy:.2f}')



/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


2024-09-23 20:08:46.981707: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 133434000 exceeds 10% of free system memory.


13238/13238 ━━━━━━━━━━━━━━━━━━━━ 95s 7ms/step - accuracy: 0.8017 - loss: 0.6814 - val_accuracy: 0.8220 - val_loss: 0.6272
Epoch 2/50
13238/13238 ━━━━━━━━━━━━━━━━━━━━ 92s 7ms/step - accuracy: 0.8148 - loss: 0.6474 - val_accuracy: 0.8233 - val_loss: 0.6251
Epoch 3/50
13238/13238 ━━━━━━━━━━━━━━━━━━━━ 93s 7ms/step - accuracy: 0.8145 - loss: 0.6482 - val_accuracy: 0.8234 - val_loss: 0.6239
Epoch 4/50
13238/13238 ━━━━━━━━━━━━━━━━━━━━ 92s 7ms/step - accuracy: 0.8147 - loss: 0.6474 - val_accuracy: 0.8235 - val_loss: 0.6245
Epoch 5/50
13238/13238 ━━━━━━━━━━━━━━━━━━━━ 92s 7ms/step - accuracy: 0.8149 - loss: 0.6469 - val_accuracy: 0.8232 - val_loss: 0.6258
Epoch 6/50
13238/13238 ━━━━━━━━━━━━━━━━━━━━ 92s 7ms/step - accuracy: 0.8156 - loss: 0.6439 - val_accuracy: 0.8232 - val_loss: 0.6257
5430/5430 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.8134 - loss: 0.6489
Test Loss: 0.65, Test Accuracy: 0.81


In [1]:
%%writefile Optimize.py
def create_model(units=50, dropout_rate=0.2, batch_size=32, epochs=50, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, input_shape=(time_steps, X.shape[2]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0, dropout_rate=0.2, units=50, optimizer='adam')

param_grid = {
    'units': [50, 100],
    'dropout_rate': [0.2, 0.3],
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 40],
    'epochs': [30, 50]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

best_model = grid_result.best_estimator_.model
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.2f}, Test Accuracy: {accuracy:.2f}')


Writing Optimize.py


In [5]:
def predict_real_time(model, new_data, lead_time=10):
    scaled_new_data = scaler.transform(new_data)
    sequences = []
    for i in range(len(scaled_new_data) - time_steps - lead_time + 1):
        sequences.append(scaled_new_data[i:(i + time_steps)])
    sequences = np.array(sequences)
    predictions = model.predict(sequences)
    return predictions


In [6]:
from tensorflow.keras.models import save_model
save_model(model, 'lstm_model.keras')

In [7]:
model.export("export/Servo/1")

INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


Saved artifact at 'export/Servo/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10, 6), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  140371869999600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140371869997488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140371870001712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140371869766000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140371869767408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140371869769168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140371869776560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140371869777264: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [8]:
import tarfile

with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("export")

In [9]:
s3_response = sm_session.upload_data("model.tar.gz", bucket=bucket_name, key_prefix="model")


In [10]:
%%writefile train.py
import os
import numpy as np
import tensorflow as tf
import joblib
import json

def model_fn(model_dir):
    model_path = os.path.join(model_dir, 'export', 'Servo', '1', 'lstm_model.keras')
    model = tf.keras.models.load_model(model_path)
    return model

def input_fn(request_body, content_type, time_steps=10):
    if content_type == 'application/json':
        request = json.loads(request_body)
        data = pd.DataFrame(request)
    elif content_type == 'text/csv':
        data = pd.read_csv(io.StringIO(request_body), header=None)
    elif content_type == 'application/x-npy':
        stream = BytesIO(request_body)
        data = np.load(stream)
        data = pd.DataFrame(data)
    else:
        raise ValueError(f'Unsupported content type: {content_type}')
    X = []
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:(i + time_steps)].values) 
    return np.array(X)

def predict_fn(input_data, model):
    predictions = model.predict(input_data) 
    return predictions

def output_fn(prediction, content_type):
    response = prediction.tolist()
    return json.dumps(response)



Overwriting train.py


In [11]:
sagemaker_model = TensorFlowModel(
    model_data=f"s3://{bucket_name}/model/model.tar.gz",
    role=role,
    framework_version="2.16",
    entry_point= 'train.py'
)

In [12]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

-----!CPU times: user 297 ms, sys: 47.7 ms, total: 345 ms
Wall time: 3min 2s


In [15]:
endpoint='tensorflow-inference-2024-09-23-20-20-30-511'
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint, sm_session)

In [16]:
result=predictor.predict(np.array([[
    [85, 18, 140, 90, 37.0, 98],
    [88, 18, 142, 92, 37.1, 97],
    [90, 19, 145, 94, 37.2, 96],
    [92, 19, 147, 95, 37.3, 95],
    [94, 20, 150, 97, 37.4, 94],
    [96, 20, 152, 98, 37.5, 93],
    [98, 21, 155, 100, 37.6, 92],
    [100, 21, 157, 102, 37.7, 91],
    [102, 22, 160, 103, 37.8, 90],
    [104, 22, 162, 105, 37.9, 89]
]]))
print(result)

{'predictions': [[0.11210607, 0.0232726652, 0.037591733, 0.827029586]]}


In [19]:
def predict_real_time(model, new_data, lead_time=10):
    sequences = []
    for i in range(len(new_data) - time_steps - lead_time + 1):
        sequences.append(new_data[i:(i + time_steps)])
    sequences = np.array(sequences)
    predictions = model.predict(sequences)
    return predictions
predict_real_time(predictor,new_patient_data, lead_time=10)

{'predictions': [[0.11210604, 0.0232726596, 0.0375917293, 0.827029526],
  [0.108835287, 0.0232997891, 0.0373404324, 0.830524504],
  [0.104884014, 0.0234275758, 0.037259344, 0.834429085],
  [0.104352511, 0.0220787786, 0.0340504237, 0.839518368],
  [0.102130376, 0.0215444732, 0.035568729, 0.840756416],
  [0.102318302, 0.0216496885, 0.0358752161, 0.840156794],
  [0.10213802, 0.0218241308, 0.0362785, 0.83975935],
  [0.102666594, 0.0219462961, 0.0363970064, 0.838990211],
  [0.103227541, 0.0220625456, 0.0365837589, 0.838126063],
  [0.105437182, 0.0223772824, 0.0367264301, 0.835459173],
  [0.132447973, 0.0302486159, 0.0376081578, 0.799695253],
  [0.171072423, 0.0406888835, 0.0381986164, 0.750040054],
  [0.191057146, 0.040819522, 0.0384506918, 0.729672551],
  [0.205713317, 0.0398204289, 0.0387327261, 0.715733588],
  [0.214534834, 0.0397323556, 0.0389994, 0.706733346],
  [0.222597122, 0.0397190973, 0.0390701629, 0.698613644],
  [0.227318063, 0.0396663137, 0.0392571799, 0.693758309],
  [0.231205

In [20]:
def predict_real_time(model, new_data, lead_time=10):
    scaled_new_data = scaler.transform(new_data)
    sequences = []
    for i in range(len(scaled_new_data) - time_steps - lead_time + 1):
        sequences.append(scaled_new_data[i:(i + time_steps)])
    sequences = np.array(sequences)
    predictions = model.predict(sequences)
    return predictions
predict_real_time(predictor,new_patient_data, lead_time=10)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


{'predictions': [[0.0296158772, 0.148461133, 0.0191821288, 0.802740812],
  [0.0313914865, 0.169540748, 0.0207802542, 0.77828759],
  [0.0337662, 0.192175627, 0.0219418947, 0.752116323],
  [0.0375614613, 0.221397132, 0.0230047032, 0.718036771],
  [0.0433702022, 0.260551214, 0.0236050449, 0.672473669],
  [0.0499644503, 0.305658132, 0.0240579862, 0.620319426],
  [0.0569920316, 0.35484919, 0.0242628232, 0.563896],
  [0.0651943609, 0.414125085, 0.0241511874, 0.496529311],
  [0.0762244165, 0.483157188, 0.0231686514, 0.417449743],
  [0.0886544511, 0.552740693, 0.0216104165, 0.33699438],
  [0.100148931, 0.60572958, 0.0202674102, 0.273854047],
  [0.105498917, 0.634870648, 0.0199435558, 0.239686772],
  [0.109200336, 0.656055748, 0.0198906455, 0.214853302],
  [0.118686, 0.672493279, 0.019504264, 0.189316452],
  [0.137719646, 0.680163622, 0.01821783, 0.163898796],
  [0.181736767, 0.665857911, 0.016491618, 0.135913655],
  [0.281589359, 0.595623493, 0.0152968476, 0.107490189],
  [0.417899609, 0.47792

In [18]:
import numpy as np
new_patient_data = np.array([
    [85, 18, 140, 90, 37.0, 98],
    [88, 18, 142, 92, 37.1, 97],
    [90, 19, 145, 94, 37.2, 96],
    [92, 19, 147, 95, 37.3, 95],
    [94, 20, 150, 97, 37.4, 94],
    [96, 20, 152, 98, 37.5, 93],
    [98, 21, 155, 100, 37.6, 92],
    [100, 21, 157, 102, 37.7, 91],
    [102, 22, 160, 103, 37.8, 90],
    [104, 22, 162, 105, 37.9, 89],
    [106, 23, 165, 107, 38.0, 88],
    [108, 23, 167, 108, 38.1, 87],
    [110, 24, 170, 110, 38.2, 86],
    [112, 24, 172, 112, 38.3, 85],
    [114, 25, 175, 113, 38.4, 84],
    [116, 25, 177, 115, 38.5, 83],
    [118, 26, 180, 117, 38.6, 82],
    [120, 26, 182, 118, 38.7, 81],
    [122, 27, 185, 120, 38.8, 80],
    [124, 27, 187, 122, 38.9, 79],
    [126, 28, 190, 123, 39.0, 78],
    [128, 28, 192, 125, 39.1, 77],
    [130, 29, 195, 127, 39.2, 76],
    [132, 29, 197, 128, 39.3, 75],
    [134, 30, 200, 130, 39.4, 74],
    [136, 30, 202, 132, 39.5, 73],
    [138, 31, 205, 133, 39.6, 72],
    [140, 31, 207, 135, 39.7, 71],
    [142, 32, 210, 137, 39.8, 70],
    [144, 32, 212, 138, 39.9, 69],
    [146, 33, 215, 140, 40.0, 68],
    [148, 33, 217, 142, 40.1, 67],
    [150, 34, 220, 143, 40.2, 66],
    [152, 34, 222, 145, 40.3, 65],
    [154, 35, 225, 147, 40.4, 64],
    [156, 35, 227, 148, 40.5, 63],
    [158, 36, 230, 150, 40.6, 62],
    [160, 36, 232, 152, 40.7, 61],
    [162, 37, 235, 153, 40.8, 60],
    [164, 37, 237, 155, 40.9, 59],
    [166, 38, 240, 157, 41.0, 58],
    [168, 38, 242, 158, 41.1, 57],
    [170, 39, 245, 160, 41.2, 56],
    [172, 39, 247, 162, 41.3, 55],
    [174, 40, 250, 163, 41.4, 54],
    [176, 40, 252, 165, 41.5, 53],
    [178, 41, 255, 167, 41.6, 52],
    [180, 41, 257, 168, 41.7, 51],
    [182, 42, 260, 170, 41.8, 50]
])


In [13]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test_classes, y_pred_classes)
print('Confusion Matrix:\n', cm)

report = classification_report(y_test_classes, y_pred_classes, target_names=['No Event', 'Heart Attack', 'Hyperglycemia', 'Hypoglycemia'])
print('Classification Report:\n', report)


  20/5430 [..............................] - ETA: 28s

2024-09-04 11:34:24.026988: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 41698320 exceeds 10% of free system memory.


5430/5430 [==============================] - 29s 5ms/step
Confusion Matrix:
 [[78664  4198  4516  4548]
 [ 4328 19416  1003   841]
 [ 4550   974 21340  1023]
 [ 4596   926   981 21839]]
Classification Report:
                precision    recall  f1-score   support

     No Event       0.85      0.86      0.85     91926
 Heart Attack       0.76      0.76      0.76     25588
Hyperglycemia       0.77      0.77      0.77     27887
 Hypoglycemia       0.77      0.77      0.77     28342

     accuracy                           0.81    173743
    macro avg       0.79      0.79      0.79    173743
 weighted avg       0.81      0.81      0.81    173743

